#### Постановка задачи
Проблема прогнозирования спроса клиентов является одной из ключевых во всех сегментах рынка. Любая компания заинтересована в увеличении реализации, привлечения и удержания своих клиентов. Один из ключевых факторов является конечная цена клиента. Постановка задачи: определить влияние изменения цены для корпоративного клиента на его потребление (НП).

#### Цель
Разработать алгоритм, определяющий количество приобретенного клиентом нефтепродукта за период (январь 2018- 15 марта 2018) для конкретного клиента (CLIENT), в разрезе транзакции в зависимости от его конечной цены и его сегмента.

#### Данные для разработки модели
Участникам предоставляется обезличенная выгрузка транзакционных данных клиентов «Газпромнефть» по региону, и нефтепродукту. Выгрузка содержит данные о покупках клиентов за период ноябрь 2016 по 15 марта 2018 года (причем за период с 1 января 2018 по 15 марта 2018 данные НЕ включают объемы).

#### Общее описание подхода
В файлах необходимо заполнить колонки «COL_LITR» напротив каждого клиента (CLIENT) для транзакций с 1 января по 15 марта 2018г, а также сегментировать клиентов по выборке (см. внизу разбивку), то есть добавить дополнительный столбец. Преимущество будет отдано той команде, которая кроме выполнения основного задания сможет дать динамику коммерческого оттока и коэффициент удержания клиентов, а также дополнительно дать информацию по поведению клиентов в зависимости от сегмента.

#### Формат предоставления результатов
Результаты необходимо предоставить в файлах: * Табличный (обязательно!) * Графический * другой

#### Оценка результатов моделирования
Результаты оцениваются путем сравнения предсказанных значений с фактическим поведением клиента (оценивает Заказчик).

In [3]:
import pandas as pd
import numpy as np

import time, json, datetime
import urllib.request as ur 

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

from scipy import sparse

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import lightgbm as lgb

import matplotlib.pyplot as plt
%matplotlib inline  

In [4]:
!ls data

sample_submission.csv  test_extended.csv  train_extended.csv
test_data.csv	       train_data.csv


### Load data

In [5]:
dtype = {'tm_wday' :np.int8, 'COL_LITR': np.float32, 'CENA_CLIENT': np.float32,
         'tm_mon': np.int8, 'tm_mday': np.int8, 'tm_hour': np.int8, 'tm_dist': np.int8,
         'REGION_AZS': np.int8, 'VID_NP': np.int8, 'holiday': np.int8}

train = pd.read_csv('data/train_extended.csv', dtype = dtype)
test = pd.read_csv('data/test_extended.csv', dtype = dtype)
sample = pd.read_csv('data/sample_submission.csv', index_col = 'ID')

In [6]:
train.head()

,CLIENT,AZS_NUMBER,REGION_AZS,VID_NP,COL_LITR,CENA_CLIENT,holiday,tm_year,tm_mon,tm_mday,tm_wday,tm_hour,tm_dist
0,46219,2335,3,1,15.000000,35.700001,0,2016,11,1,1,0,0
1,33792,3356,18,2,30.000000,31.440001,0,2016,11,1,1,1,0
2,16215,3381,18,1,220.000000,32.939999,0,2016,11,1,1,3,0
3,33792,3303,18,2,15.000000,31.160000,0,2016,11,1,1,1,0
4,17075,3276,18,1,133.600006,34.720001,0,2016,11,1,1,2,0


### Feature extraction

In [7]:
def holidays(d, m, year):
    time.sleep(0.2)
    elevations = ur.urlopen("https://kayaposoft.com/enrico/json/v2.0/?action=isPublicHoliday&date="+str(d)+'-'+str(m)+'-'+str(year)+"&country=ru").read()
    data = json.loads(elevations)
    if data['isPublicHoliday'] == True:
        return 1
    else:
        return 0

def extr(df):
    df = df.set_value(df[pd.isnull(df['DATA_TRANS'])].index, 'DATA_TRANS', '11.11.2011 1:1:1')
    
    df['d_m_year'] = df['DATA_TRANS'].progress_apply(lambda x: x.split()[0])
    tmp = pd.DataFrame([[holidays(d.split('.')[0], d.split('.')[1], d.split('.')[2]), d] for d in tqdm_notebook(df['d_m_year'].unique())], columns=['holiday', 'd_m_year'])
    df = df.merge(tmp, on='d_m_year')
    
    df['holiday'].to_csv('holiday.csv')
    return 0
    
    del df['d_m_year']
    del tmp 
    
    
    df['check'] = df['DATA_TRANS'].progress_apply(lambda x: 1 if len(x.split()) != 1 else 0)
        
    df[df['check'] == 1]['tm_year'] = df[df['check'] == 1]['DATA_TRANS'].progress_apply(lambda x: time.strptime(str(x), '%d.%m.%Y %H:%M:%S')[0])
    df[df['check'] == 1]['tm_mon'] = df[df['check'] == 1]['DATA_TRANS'].progress_apply(lambda x:  time.strptime(str(x), '%d.%m.%Y %H:%M:%S')[1])
    df[df['check'] == 1]['tm_mday'] = df[df['check'] == 1]['DATA_TRANS'].progress_apply(lambda x: time.strptime(str(x), '%d.%m.%Y %H:%M:%S')[2])
    df[df['check'] == 1]['tm_wday'] = df[df['check'] == 1]['DATA_TRANS'].progress_apply(lambda x: time.strptime(str(x), '%d.%m.%Y %H:%M:%S')[6])
    
    df[df['check'] == 1]['tm_hour'] = df[df['check'] == 1]['DATA_TRANS'].progress_apply(lambda x: time.strptime(str(x), '%d.%m.%Y %H:%M:%S')[3])
    df[df['check'] == 1]['tm_dist'] = df[df['check'] == 1].progress_apply(lambda row: 0 if row['tm_hour'] <= 6 or row['tm_hour'] == 23 else 1, axis = 1)
    df[df['check'] == 1]['tm_dist'] = df[df['check'] == 1]['DATA_TRANS'].progress_apply(lambda x: time.strptime(str(x), '%d.%m.%Y %H:%M:%S')[8])
    
    
    del df['check']
    del df['DATA_TRANS']
    return df


In [8]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)
train = clean_dataset(train)
test = clean_dataset(test)

train = extr(train)

test = extr(test)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(train[[c for c in train.columns if c != 'COL_LITR']].astype(np.float64).values, 
                                                    train['COL_LITR'].astype(np.float64).values,
                                                    test_size=0.35, random_state=42)

### Predict

In [ ]:
def Test_lgb(X_train, y_train):
    clf = lgb.LGBMRegressor(n_jobs=1)
    
    param = {
    'learning_reate': [0.1,0.2, 0.001, 0.02],
    'num_leaves': [31, 127],
    'feature_fraction': [0.5, 1.0],
    'bagging_fraction': [0.75, 0.95], 
    'reg_alpha': [0.1, 0.5],
    'n_estimators': [20, 50]
    }
    
    start_time = datetime.datetime.now()
    gs = GridSearchCV(clf, scoring='neg_mean_squared_error', param_grid=param, cv=4, 
                      return_train_score=True, n_jobs=-1, verbose = True)
    
    gs.fit(X_train,y_train)
    
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    print (max(gs.cv_results_['mean_test_score']))
    
    return gs.best_params_

In [1]:
#Test_lgb(train[[c for c in train.columns if c != 'COL_LITR']].astype(np.float64).values, train['COL_LITR'].astype(np.float64).values)

In [11]:
gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=5)


[1]	valid_0's l1: 70.4653
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 68.9862
[3]	valid_0's l1: 67.6094
[4]	valid_0's l1: 66.3453
[5]	valid_0's l1: 65.1358
[6]	valid_0's l1: 64.045
[7]	valid_0's l1: 63.0392
[8]	valid_0's l1: 62.091
[9]	valid_0's l1: 61.1852
[10]	valid_0's l1: 60.3375
[11]	valid_0's l1: 59.5585
[12]	valid_0's l1: 58.8415
[13]	valid_0's l1: 58.1666
[14]	valid_0's l1: 57.5587
[15]	valid_0's l1: 56.9666
[16]	valid_0's l1: 56.4249
[17]	valid_0's l1: 55.9193
[18]	valid_0's l1: 55.4381
[19]	valid_0's l1: 54.9839
[20]	valid_0's l1: 54.5723
Did not meet early stopping. Best iteration is:
[20]	valid_0's l1: 54.5723


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.05, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=20,
       n_jobs=-1, num_leaves=31, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

In [ ]:
sample['COL_LITR'] = gbm.predict(test.values)

In [ ]:
sample.to_csv('lgb_result.csv')